_Supervised learning for classification_

### 4.1 Import data

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import plotly.graph_objects as go
from sklearn.metrics import (accuracy_score, 
                             precision_score, 
                             recall_score, 
                             f1_score, 
                             confusion_matrix,
                             roc_curve,
                             auc)
import joblib
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
# import ace_tools as tools
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

with open("outputs/03_Variables.pkl", 'rb') as file:
    (X_train, X_test, y_train, y_test, kfold) = pickle.load(file)

model_logistic_regression = joblib.load('outputs/03-1_Logistic_Regression_final_model.joblib')
model_decision_tree = joblib.load('outputs/03-2_Decision_Tree_final_model.joblib')
model_random_forest = joblib.load('outputs/03-3_Random_Forest_final_model.joblib')
model_SVM = joblib.load('outputs/03-4_SVM_final_model.joblib')

trained_models = {"Logistic Regression": model_logistic_regression,
                  "Decision Tree": model_decision_tree,
                  "Random Forest": model_random_forest,
                  "SVM": model_SVM}

### 4.2 Benchmark the trained models

In [ ]:
# Create lists to store evaluation metrics for each model
model_names = []
accuracies = []
precisions = []
recalls = []
f1_scores = []
conf_matrices = []
misclassified_samples = {}

# Loop through each trained model
for model_name in trained_models:
    # Make predictions on the test dataset
    y_pred = trained_models[model_name].predict(X_test)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Append metrics to lists
    model_names.append(model_name)
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)
    
    # Find indices of misclassified samples
    misclassified_indices = [i for i, (true, pred) in enumerate(zip(y_test, y_pred)) if true != pred]
    
    # Store misclassified samples with their true and predicted labels
    misclassified_samples[model_name] = {
        'sample_names': X_test.index[misclassified_indices],  # Assuming X_test is a DataFrame
        'true_label': np.array(y_test)[misclassified_indices],
        'predicted_label': y_pred[misclassified_indices]
    }

# Create a DataFrame to store the evaluation results
evaluation_df = pd.DataFrame({
    'Model': model_names,
    'Accuracy': accuracies,
    'Precision': precisions,
    'Recall': recalls,
    'F1 Score': f1_scores,
    'Confusion Matrix': conf_matrices
})

# Display the evaluation results
display(evaluation_df.drop(columns="Confusion Matrix"))

In [ ]:
# Create the evaluation metrics bar chart using Plotly
fig = go.Figure()

fig.add_trace(go.Bar(x=model_names, y=accuracies, name='Accuracy'))
fig.add_trace(go.Bar(x=model_names, y=f1_scores, name='F1 Score'))
fig.add_trace(go.Bar(x=model_names, y=precisions, name='Precision'))
fig.add_trace(go.Bar(x=model_names, y=recalls, name='Recall'))

fig.update_layout(title='Model Evaluation Metrics',
                  xaxis_title='Model',
                  yaxis_title='Score',
                  barmode='group')

fig.show()

#### List of mislabeled samples

In [ ]:
misclassified_df = pd.DataFrame(columns=['Model', 'Sample Name', 'True Label', 'Predicted Label'])

# Loop through each model and collect misclassified sample information
for model_name in misclassified_samples:
    misclassified = misclassified_samples[model_name]
    
    # Create a temporary DataFrame for the current model's misclassified samples
    temp_df = pd.DataFrame({
        'Model': model_name,  # Add model name to each row
        'Sample Name': misclassified['sample_names'],
        'True Label': misclassified['true_label'],
        'Predicted Label': misclassified['predicted_label']
    })
    
    # Append to the main DataFrame
    misclassified_df = pd.concat([misclassified_df, temp_df], ignore_index=True)

# Display the resulting DataFrame with misclassified samples
display(misclassified_df)

### 4.3 Confusion matrices

In [ ]:
# Assume class_labels and conf_matrices are defined
class_labels = np.unique(y_train)

# Create a subplot grid to plot multiple confusion matrices (2 rows for aesthetics)
n_models = len(trained_models)
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(6, 13))

# Flatten the axes array to easily iterate (since we may have fewer models than subplot slots)
axes = axes.flatten()

# Plot each confusion matrix using a heatmap
for i, model_name in enumerate(trained_models):
    sns.heatmap(conf_matrices[i], annot=True, cmap='Blues', fmt='g', ax=axes[i])
    axes[i].set_title(model_name)
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('True')
    
    # Set the correct number of ticks based on the number of classes
    axes[i].set_xticks(np.arange(len(class_labels)))
    axes[i].set_yticks(np.arange(len(class_labels)))
    
    # Set the class labels as tick labels
    axes[i].set_xticklabels(class_labels, rotation=45, ha='right', fontsize=10)
    axes[i].set_yticklabels(class_labels, rotation=0, fontsize=10)

# Remove unused subplots if necessary
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## ROC

In [ ]:
# %%ignore
# Create the ROC curve plot using Plotly
# For single class
# fig = go.Figure()

# for model_name in trained_models:
#     trained_models[model_name].fit(X_train, y_train)
#     y_pred_prob = trained_models[model_name].predict_proba(X_test)[:, 1]
#     fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
#     roc_auc = auc(fpr, tpr)
    
#     # Add ROC curve trace for each classifier
#     fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'{model_name} (AUC = {roc_auc:.2f})'))

# # Add diagonal line representing random guessing
# fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(color='gray', dash='dash'), name='Random Guessing'))

# fig.update_layout(title='ROC Curve Comparison for Different Classifiers',
#                   xaxis_title='False Positive Rate',
#                   yaxis_title='True Positive Rate',
#                   legend_title='Classifiers',
#                   showlegend=True)

# fig.show()

In [ ]:
# Create the ROC curve plot using Plotly
# For multiple classes
for model_name in trained_models:
    fig = go.Figure()
    trained_models[model_name].fit(X_train, y_train)
    # Get the probabilities for each class
    y_prob = trained_models[model_name].predict_proba(X_test)
    # Calculate ROC curve and AUC for each class
    for i in range(len(trained_models[model_name].classes_)):
        fpr, tpr, _ = roc_curve(y_test == trained_models[model_name].classes_[i], y_prob[:, i])
        roc_auc = auc(fpr, tpr)
        
        # Add ROC curve trace for each class
        fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', 
                                 name=f'{model_name} (Class {trained_models[model_name].classes_[i]}) (AUC = {roc_auc:.2f})'))

    # Add diagonal line representing random guessing
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(color='gray', dash='dash'), name='Random Guessing'))

    fig.update_layout(title='ROC for '+model_name+' (Multi-Class)',
                    xaxis_title='False Positive Rate',
                    yaxis_title='True Positive Rate',
                    xaxis_range=[0,1],yaxis_range=[0,1],
                    legend_title='Classifiers',
                    showlegend=True,
                    plot_bgcolor='white', paper_bgcolor='white',
                    xaxis=dict(linecolor='black'),
                    yaxis=dict(linecolor='black'),
                    legend=dict(orientation="h",
                                yanchor="top",y=-0.2,
                                xanchor="left",x=-0.2),
                    height=500, width=500)
    fig.show()